In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, classification_report

from imblearn.under_sampling import RandomUnderSampler

# 1. LOAD DATA
train = pd.read_csv("BinaryTrain.csv")
test = pd.read_csv("TestBinary.csv")

TARGET = "retention_status"
ID_COL = "founder_id"

X = train.drop(columns=[TARGET, ID_COL])
y = train[TARGET]

X_test = test.drop(columns=[ID_COL], errors="ignore")

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 2. PREPROCESSING
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

numeric = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())          # REQUIRED for KNN
])

categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer([
    ("num", numeric, num_cols),
    ("cat", categorical, cat_cols)
])

# Preprocess full training data
X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test)

# 3. BALANCING (Optional but helpful for KNN)

rus = RandomUnderSampler(random_state=42)
X_bal, y_bal = rus.fit_resample(X_processed, y_encoded)

# 4. TRAIN/VALID SPLIT

X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(
    X_bal, y_bal, test_size=0.2, random_state=42, stratify=y_bal
)

# 5. HYPERPARAMETER TUNING FOR KNN
knn = KNeighborsClassifier()

param_dist = {
    "n_neighbors": [3, 5, 7, 9, 11, 15],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "minkowski"],
    "p": [1, 2]   # p=1 L1 distance, p=2 L2 distance
}

search = RandomizedSearchCV(
    knn,
    param_distributions=param_dist,
    n_iter=12,
    scoring="f1_weighted",
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("🔍 Running KNN hyperparameter search...")
search.fit(X_train_s, y_train_s)

best_knn = search.best_estimator_

print("\nBest Params:", search.best_params_)
print("Best CV F1:", search.best_score_)

# 6. THRESHOLD TUNING

# probability of class=1:
val_probs = best_knn.predict_proba(X_val_s)[:, 1]  

best_f1 = 0
best_thresh = 0.5

for thr in np.linspace(0.2, 0.8, 41):
    preds = (val_probs >= thr).astype(int)
    score = f1_score(y_val_s, preds, average="weighted")

    if score > best_f1:
        best_f1 = score
        best_thresh = thr

print("\nBest Threshold:", best_thresh)
print("Best F1 Score:", best_f1)

# 7. FINAL TRAIN ON FULL BALANCED DATA

best_knn.fit(X_bal, y_bal)

# 8. TEST SET PREDICTIONS

test_probs = best_knn.predict_proba(X_test_processed)[:, 1]
test_preds = (test_probs >= best_thresh).astype(int)

test_labels = le.inverse_transform(test_preds)

# 9. SAVE SUBMISSION
submission = pd.DataFrame({
    "founder_id": test["founder_id"],
    "retention_status": test_labels
})

submission.to_csv("submission_knn_optimized.csv", index=False)
print("\nSaved: submission_knn_optimized.csv")
print(submission.head())